## Векторизация наводнений на ходу по проектной территории из tiff

In [ ]:
from ecodonut.landscape_vectorizer import vectorize_flood_depth
from pathlib import Path
import geopandas as gpd

tiff_dir = Path("D:\ItmoProjects\EcoDonut\dev\GloFAS_flood_tiles")  # папка с GloFAS_flood тайлами
zone_gdf = gpd.read_file("example_data/project Шлиссельбург.geojson")
tiles_gdf = gpd.read_parquet("example_data/glofas_flood_tiles_russia.parquet")

#  Literal["RP10", "RP20", "RP50", "RP75", "RP100", "RP200", "RP500"] - доступные значения повторений наводнений по годам

res = vectorize_flood_depth(tiff_dir, repeatability_label="RP500", tiles_gdf=tiles_gdf, zone_gdf=zone_gdf,step_m=0.1)

In [ ]:
res.explore(column='depth_m')

## Векторизация состава почвы на ходу по проектной территории из tiff

In [ ]:
from ecodonut.landscape_vectorizer import vectorize_soilgrid
from pathlib import Path
import geopandas as gpd

tiff_dir = Path("D://soilgrid_tiles")  # папка с soilgrid тайлами
zone_gdf = gpd.read_file("example_data/project Шлиссельбург.geojson")
tiles_gdf = gpd.read_parquet("example_data/fabdem_tiles_russia.parquet")

#  Literal["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"] - доступные высотности

res = vectorize_soilgrid(tiff_dir, depth_label="0-5cm", tiles_gdf=tiles_gdf, zone_gdf=zone_gdf, step_pct=5,
                         usda_to_russian=True)

In [ ]:
res.explore(column = 'usda_texture')

## Векторизация ландшафта на ходу по проектной территории из tiff

In [ ]:
from ecodonut.landscape_vectorizer import vectorize_height_isolines_for_zone, vectorize_height_polygons_for_zone, \
    vectorize_aspect_polygons_for_zone, vectorize_slope_polygons_for_zone
import geopandas as gpd
from pathlib import Path

zone_gdf = gpd.read_file("example_data/Территория проекта (1).geojson").to_crs(32636)
tiles_gdf = gpd.read_parquet("example_data/fabdem_tiles_russia.parquet")  # файл с тайлами
tiffs_dir = Path("D://FABDEM_WORK/fabdem_tiles_russia")  # папка с FABDEM.tiff

# iso_lines = vectorize_height_isolines_for_zone(zone_gdf=zone,tiles_gdf=tiles,tiffs_dir=tiff_dir,height_step=1,smooth_sigma=0.1)
heigh_map = vectorize_height_polygons_for_zone(zone_gdf=zone_gdf, tiles_gdf=tiles_gdf, tiffs_dir=tiffs_dir, height_step=1,
                                               smooth_sigma=0.1)
# slope_map = vectorize_slope_polygons_for_zone(zone_gdf=zone_gdf, tiles_gdf=tiles_gdf, tiffs_dir=tiffs_dir, slope_step_deg=1,
#                                               smooth_sigma=0.1)
# aspect_map = vectorize_aspect_polygons_for_zone(zone_gdf=zone_gdf, tiles_gdf=tiles_gdf, tiffs_dir=tiffs_dir, aspect_step_deg=45,
#                                                 smooth_sigma=0.1)


In [ ]:
heigh_map.to_parquet('example_data/old_height.parquet')

In [ ]:
heigh_map.to_parquet('example_data/new_height2.parquet')

In [ ]:
heigh_map.explore(column='height', tiles='cartodb positron')

In [ ]:
from matplotlib.colors import LinearSegmentedColormap


def make_aspect_cmap(name: str = "aspect360") -> LinearSegmentedColormap:
    stops = [
        (0.00, "#5b7e94"),  # N   (0°)    blue
        (0.25, "#579976"),  # E   (90°)   green
        (0.50, "#965253"),  # S   (180°)  red
        (0.75, "#595263"),  # W   (270°)  purple
        (1.00, "#5b7e94"),  # N   (360°)  blue (wrap)
    ]
    return LinearSegmentedColormap.from_list(name, stops)


cmap = make_aspect_cmap('aspect_deg')
aspect_map.to_crs(3857).plot(column='aspect_deg', cmap=cmap)

In [ ]:
m1 = heigh_map.explore(column='height', tiles='cartodb positron')
slope_map.explore(m=m1, column='slope_deg', cmap='Reds')

## Работа с векторизованными слоями по тайлам 

In [ ]:
from pathlib import Path
import geopandas as gpd
from ecodonut.landscape_vectorizer import (
    stitch_height_isolines, stitch_height_polygons,
    stitch_slope_polygons, stitch_aspect_polygons,
)

zone = gpd.read_file("example_data/project_zone.geojson")
zone.geometry = zone.to_crs(zone.estimate_utm_crs()).geometry.buffer(5000).to_crs(zone.crs)  # Буффер, если надо
tiles = gpd.read_parquet("example_data/fabdem_tiles.parquet")  # файл с тайлами
vectors_dir = Path("D://fabdem_tiles_russia")  # папка с векторизованными слоями

# 1) Изолинии
iso = stitch_height_isolines(zone, tiles, vectors_dir, height_step=5)

# 2) Полигоны высоты
highpoly = stitch_height_polygons(zone, tiles, vectors_dir, height_step=5)

# 3) Уклон
slope = stitch_slope_polygons(zone, tiles, vectors_dir, slope_step_deg=2)

# 4) Экспозиция
aspect = stitch_aspect_polygons(zone, tiles, vectors_dir, aspect_step_deg=90)

# Визуализация состава почвы

In [ ]:
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from ecodonut.landscape_vectorizer import vectorize_soilgrid

tiff_dir = Path("example_data/soilgrid_tiles")
tile_name = "N059E030"
depths = ["0-5cm", "5-15cm", "15-30cm", "30-60cm", "60-100cm", "100-200cm"]

texture_colors = {
    "Песок": "bisque",
    "Супесь песчаная": "#f4a582",
    "Супесь": "#d6604d",
    "Суглинок лёгкий": "#92c5de",
    "Суглинок пылеватый": "#4393c3",
    "Пылеватый грунт (ил)": "#2166ac",
    "Суглинок тяжёлый песчанистый": "#b2df8a",
    "Суглинок тяжёлый": "#33a02c",
    "Суглинок тяжёлый пылеватый": "#1b7837",
    "Глина песчанистая": "#cab2d6",
    "Глина пылеватая": "#6a3d9a",
    "Глина": "#542788",
}

depth_to_gdf: dict[str, gpd.GeoDataFrame] = {}

for depth in depths:
    print(f"[compute] depth={depth}")
    gdf = vectorize_soilgrid(
        tiff_dir,
        depth_label=depth,
        tile_name=tile_name,
        step_pct=5,
        usda_to_russian=True,
    )
    if not gdf.empty and gdf.crs != zone.crs:
        gdf = gdf.to_crs(zone.crs)
    depth_to_gdf[depth] = gdf




In [ ]:


fig, axes = plt.subplots(
    2, 3,
    figsize=(8, 12),
    sharex=True,
    sharey=True,
)
axes = axes.ravel()

fig.subplots_adjust(wspace=0.0, hspace=0.0)

for ax, depth in zip(axes, depths):
    ax.set_axis_off()
    gdf = depth_to_gdf.get(depth)

    if gdf is None or gdf.empty:
        ax.set_title(f"{depth} (нет данных)", fontsize=10)
        continue

    tex = gdf["usda_texture"].astype(str)
    colors = tex.map(texture_colors).fillna("#cccccc")

    gdf.to_crs(3857).plot(ax=ax, color=colors, linewidth=0)

    ax.set_title(depth, fontsize=10)

plt.tight_layout(rect=[0.0, 0.0, 1.0, 0.88])

legend_elems = [
    Line2D(
        [0], [0],
        marker="s",
        linestyle="none",
        markersize=10,
        markerfacecolor=col,
        markeredgecolor="none",
        label=lab,
    )
    for lab, col in texture_colors.items()
]

fig.legend(
    handles=legend_elems,
    loc="upper center",
    bbox_to_anchor=(0.5, 0.98),
    ncol=3,
    frameon=False,
)

plt.show()